In this example, we will study the skyrmion dynamics in a 2d film. We will save the skyrmion
positions to a text file and and generate a movie at the end of simulation.

We import the necessary modules

In [1]:
using MicroMagnetic
using Printf

@using_gpu()

set_verbose()

The studied system is a 800nm x 300nm x 2nm film with periodic boundary conditions.

In [2]:
mesh = FDMesh(; nx=400, ny=150, nz=1, dx=2e-9, dy=2e-9, dz=2e-9, pbc="xy");

We create a Sim instance using `create_sim` function, and set basic simulation parameters such as
'Ms', 'A', 'D' and 'H'.

In [3]:
sim = create_sim(mesh; Ms=3.87e5, A=5.2e-12, D=1e-3, H=(0, 0, 160 * mT), name="skx");

[ Info: MicroSim has been created.
[ Info: Exchange has been added.
[ Info: Bulk DMI has been added.
[ Info: Static Zeeman has been added.


We set the initial magnetization configuration to a skyrmion at position (200nm, 80nm) with radius 20 nm.
Note that the initialized magnetization is a roughly guessing for the skyrimon.

In [4]:
init_m0_skyrmion(sim, (200e-9, 80e-9), 2e-8)

We relax the system to obtain the skyrmion profile.

In [5]:
relax(sim; max_steps=20000, stopping_dmdt=0.01)

[ Info: Running Driver : MicroMagnetic.EnergyMinimization{Float64}.
[ Info: max_dmdt is less than stopping_dmdt=0.01 @steps=342, Done!


We save the magnetization to vtk, which can be opened using Paraview for 3D visualization.

In [6]:
MicroMagnetic.save_vtk(sim, "skx")

1-element Vector{String}:
 "skx.vts"

After obataining the skyrmion profile, we then move the skyrmion using spin transfer torques.
So we use change the driver to "LLG_STT" using the `set_driver` function. Meanwhile,
we set the relevant parameters such as alpha, beta and u.

In [7]:
set_driver(sim; driver="LLG_STT", alpha=0.05, beta=0.2, ux=-20)

[ Info: The driver LLG_STT is used!


During the simulation, we need to extract the skyrmion center, so we write a call back function
in which the skyrmion positions are computed using the `compute_guiding_center` function and
saved to a text file with append mode.

In [8]:
function call_back_fun(sim, t)
    Rx, Ry = compute_guiding_center(sim)
    open("XY.txt", "a") do f
        return write(f, @sprintf("%g  %g  %g\n", t, Rx, Ry))
    end
end

call_back_fun (generic function with 1 method)

We use the `run_sim` function to run the simulation.
after that, a jld2 file will be created and we can export it to a movie (.mp4) using the `jld2movie`.

In [9]:
if !isfile("assets/skx.mp4")
    run_sim(sim; steps=100, dt=1e-10, save_m_every=1, call_back=call_back_fun)
    jld2movie("skx.jld2"; output="assets/skx.mp4")
end

![](./assets/skx.mp4)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*